# Archived
**We are no longer using this Visual for Content Interactions. We have switched to using the graph which shows Content Interactions over time starting from 2018**


## Visualization

In [1]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(magrittr); library(zeallot); library(glue);  library(glue); library(lubridate)
    library(scales); library(RColorBrewer); library(dplyr); library(tidyr); library(ggplot2)
    library(ggthemes); library(polloi)
})

In [2]:
file <- "metrics/metrics.tsv"
pageview <- read.csv(file, sep = '\t')
pageview$month <- as.Date(pageview$month, format = "%Y-%m-%d")

In [3]:
head(pageview)

,month,automated_pageviews,desktop,gs_interactions,gs_pageviews,gs_previews,interactions,mh_interactions,mh_pageviews,mh_previews,mobileweb,previews_seen,total_pageview,unique_devices
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2016-07-01,NA,7319709535,NA,NA,NA,NA,NA,NA,NA,6951477838,NA,14479097793,NA
2,2016-08-01,NA,7808198322,NA,NA,NA,NA,NA,NA,NA,7158465508,NA,15164063733,NA
3,2016-09-01,NA,8015803318,NA,3509314705,NA,NA,NA,482166702,NA,7037821889,NA,15227361307,NA
4,2016-10-01,NA,8597813074,NA,3649168724,NA,NA,NA,584002147,NA,7454268064,NA,16205992870,NA
5,2016-11-01,NA,8756368629,NA,3682761207,NA,NA,NA,639332261,NA,7568847051,NA,16499874726,NA
6,2016-12-01,NA,8140608026,NA,3397813382,NA,NA,NA,566587960,NA,7623805561,NA,15964600507,NA


In [4]:
tail(pageview)

,month,automated_pageviews,desktop,gs_interactions,gs_pageviews,gs_previews,interactions,mh_interactions,mh_pageviews,mh_previews,mobileweb,previews_seen,total_pageview,unique_devices
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
67,2022-01-01,1003626771,6657032289,4237581175,3860723508,376857667,19401297429,NA,NA,NA,10371244452,2058829668,17342467761,1775062017
68,2022-02-01,979406965,6381498663,4110316297,3767028146,343288151,17874089603,NA,NA,NA,9383265529,1833566161,16040523442,1827453153
69,2022-03-01,1000022585,6820776330,4407956608,4051236286,356720322,18950017723,NA,NA,NA,9870968780,1967056255,16982961468,1957129624
70,2022-04-01,1257069208,6517582524,4023481357,3700705877,322775480,17930780703,NA,NA,NA,9357139747,1773846791,16156933912,1878850994
71,2022-05-01,1491689147,6701587010,4102891003,3771585115,331305888,18174941642,NA,NA,NA,9402733691,1788004827,16386936815,1948874399
72,2022-06-01,1522578765,6287649654,3923352975,3613205737,310147238,17104956625,NA,NA,NA,8918872260,1626137883,15478818742,2056714340


### Pageviews Interactions 

In [5]:
pageview_fy <- pageview %>% 
     select(month,total_pageview,previews_seen) %>%
     mutate(previews_seen = replace_na(previews_seen, 0)) %>%
     mutate(interactions = total_pageview+previews_seen) %>%
     filter(month >= '2019-07-01') %>%
     mutate(fiscal_year = ifelse(month >= '2019-07-01' & month < '2020-07-01', 'FY 2019/20', ifelse(month <'2019-07-01','FY 2018/19',ifelse(month <'2021-07-01','FY 2020/21','FY 2021/22'))),
           MonthN =as.factor(format(as.Date(month),"%m")),
         Month = months(as.Date(month), abbreviate=TRUE))

pageview_fy$MonthN = factor(pageview_fy$MonthN, levels=c("07","08", "09", "10", "11", "12", "01", "02", "03", "04", "05", "06"))

In [6]:
pageview_fy %<>%
    mutate(
        dataloss_affected = case_when(
            `month` >= "2021-06-01" & `month` < "2021-11-01" ~ TRUE,
            `month` >= "2021-11-01" & `month` < "2022-02-01" ~ TRUE,
            TRUE ~ FALSE
        ),
        fiscal_year_expanded = case_when(
            dataloss_affected ~ paste(fiscal_year, "(undercounted)"),
            TRUE ~ fiscal_year
        )
    )

In [7]:
# For continuous line:
pageview_fy <- bind_rows(
        pageview_fy,
        pageview_fy %>%
            filter(`month` >= "2022-01-01" & `month` <= "2022-02-01") %>%
            mutate(fiscal_year_expanded = fiscal_year_expanded[1]) %>%
            filter(`month` == "2022-02-01")
    )

In [8]:
tail(pageview_fy)

,month,total_pageview,previews_seen,interactions,fiscal_year,MonthN,Month,dataloss_affected,fiscal_year_expanded
,<date>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<chr>,<lgl>,<chr>
32,2022-02-01,16040523442,1833566161,17874089603,FY 2021/22,02,Feb,FALSE,FY 2021/22
33,2022-03-01,16982961468,1967056255,18950017723,FY 2021/22,03,Mar,FALSE,FY 2021/22
34,2022-04-01,16156933912,1773846791,17930780703,FY 2021/22,04,Apr,FALSE,FY 2021/22
35,2022-05-01,16386936815,1788004827,18174941642,FY 2021/22,05,May,FALSE,FY 2021/22
36,2022-06-01,15478818742,1626137883,17104956625,FY 2021/22,06,Jun,FALSE,FY 2021/22
37,2022-02-01,16040523442,1833566161,17874089603,FY 2021/22,02,Feb,FALSE,FY 2021/22 (undercounted)


In [9]:
source("supplementary_files/pageview_dataloss_helper_functions.R")

In [10]:
pageviews_dataloss <- readr::read_tsv("supplementary_files/pageview_dataloss.tsv.gz", show_col_types = FALSE)

loss_bounds <- pageviews_dataloss %>%
    loss_boundaries() %>%
    bind_rows(.id = "which_loss")

loss_bounds

which_loss,lower,middle,upper
<chr>,<dbl>,<dbl>,<dbl>
loss1,0.02638790,0.03160639,0.03671803
loss2,0.05151559,0.06092645,0.06837827
none,0.00000000,0.00000000,0.00000000


In [11]:
dataloss_estimates_fy <- pageview_fy %>%
        mutate(
            which_loss = case_when(
                `month` >= "2021-06-01" & `month` < "2021-11-01" ~ "loss1",
                `month` >= "2021-11-01" & `month` < "2022-02-01" ~ "loss2",
                TRUE ~ "none"
            )
        ) %>%
        # Include Feb 2022 for continuity:
        filter(which_loss != "none" | `month` == "2022-02-01") %>%
        left_join(loss_bounds, by = "which_loss") %>%
        mutate(
            estimated_pageview_lower = total_pageview / (1 - lower),
            estimated_pageview_middle = total_pageview / (1 - middle),
            estimated_pageview_upper = total_pageview / (1 - upper),
            estimated_interactions_lower = previews_seen + estimated_pageview_lower,
            estimated_interactions_middle = previews_seen + estimated_pageview_middle,
            estimated_interactions_upper = previews_seen + estimated_pageview_upper
        )

In [12]:
options(repr.plot.width = 16, repr.plot.height = 8)

ggplot(pageview_fy, aes(x = MonthN)) +
    geom_line(
        aes(
            y = interactions, group = fiscal_year_expanded, color = fiscal_year_expanded, linetype = fiscal_year_expanded
        ),
        size = 1.5
    ) +
    geom_point(
        data = pageview_fy %>%
            filter(fiscal_year == "FY 2021/22", !(fiscal_year_expanded == "FY 2021/22 (undercounted)" & `month` == "2022-02-01")),
        aes(y = interactions, group = fiscal_year_expanded, color = fiscal_year_expanded, size = dataloss_affected),
    ) +
    geom_ribbon(
        data = dataloss_estimates_fy %>%
            filter(fiscal_year == "FY 2021/22"),
        aes(
            ymin = estimated_interactions_lower, ymax = estimated_interactions_upper,
            group = fiscal_year, fill = fiscal_year
        ),
        alpha = 0.8
    ) +
    scale_linetype_manual(values = c(
        "FY 2019/20" = "twodash",
        "FY 2020/21" = "dashed",
        "FY 2021/22" = "solid",
        "FY 2021/22 (undercounted)" = "solid"
    )) +
    scale_size_manual(
        values = c("TRUE" = 3, "FALSE" = 5),
        guide = "none"
    ) +
    scale_fill_manual(values = c(
        "FY 2021/22" = "#2a4b8d"
    )) +
    scale_color_manual(values = c(
        "FY 2019/20" = "#b1c4eb",
        "FY 2020/21" = "#3366cc",
        "FY 2021/22" = "#2a4b8d",
        "FY 2021/22 (undercounted)" = "gray50"
    )) +
    scale_x_discrete(
        "Month",
        breaks = pageview_fy$MonthN, 
        labels = pageview_fy$Month
    ) +
    scale_y_continuous(
        "Content Interactions per Month (billions)",
        labels = polloi::compress,
        limits = c(13E9, 23E9)
    ) +
    guides(
        fill = guide_legend(
            title = "Estimate", title.theme = element_text(family = "Courier", size = 17)
        ),
        linetype = guide_legend(
            label.hjust = 0
        )
    ) +
    ggthemes::theme_tufte(base_size = 16,base_family = "serif") +
    theme(
        axis.title.x = element_blank(),
        axis.title = element_text(size = 17, hjust = 0.5, vjust = -12),
        axis.text.x = element_text(hjust = 0, size = 17, family = "Courier"),
        axis.text.y = element_text(hjust = 0, size = 21, family = "Courier"),
        panel.grid = element_line("gray95"),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.y = element_blank(),
        plot.title = element_text(size = 20, hjust = 0.0, face = "bold"),
        legend.position = "right", 
        legend.title = element_blank(),
        legend.text = element_text(hjust = 0.5, size = 13),
        legend.key.width = unit(1.5, "cm")
    )

In [13]:
# Removed Interactions Bar - line chart as it is not used anymore